In [135]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from point2mn import point2mn
from point2mn import get_main_points
torch.manual_seed(1)

In [136]:
import numpy as np
import pandas as pd

In [137]:
embedding_index = {}

f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

print('found word vecs: ',len(embedding_index))

found word vecs:  400000


In [138]:
main_points =pd.read_csv('../Meta Data/metadata_appended_main_points.csv') 
main_points.head()

,id,title,date,for,against,For_Main_Points,against_Main_Points
0,d20191112,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...
1,d20191029,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']","[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia..."
2,d20191022,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....
3,d20190917,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",['The United States government should follow t...,['Individuals should have the freedom to choos...
4,d20190912,Unresolved: Shifting Power in the Middle East,2019-09-12,"['Michael Doran', 'Reuel Marc Gerecht', 'Berna...","['Brett McGurk', 'Barbara Slavin']",NaN,NaN


In [139]:
main_points.dropna(subset = ['For_Main_Points', 'against_Main_Points'], inplace = True)

In [140]:
main_points = main_points.reset_index(drop = True)

In [141]:
fid = 'd20191112'

In [142]:
def run_lstm_one_sentence(sentence, title, embedding_index):
    inputs = point2mn(sentence, title, embedding_index)
    dim = inputs.shape[1]
    lstm = nn.LSTM(dim, dim)
    tensor_input = [torch.tensor([x]) for x in inputs]
    tensor_input = torch.cat(tensor_input).view(len(tensor_input), 1, -1)
    hidden = (torch.randn(1, 1, dim), torch.randn(1, 1, dim))
    tensor_output, hidden = lstm(tensor_input, hidden)
    return tensor_output[-1]

In [143]:
"""
    -combine_func: the funtion applied to combine lstm outputs from one side elementwisely
"""
def run_lstm_on_fid(fid, embedding_index, combine_func = torch.min):
    title = main_points[main_points['id'] == fid].title[0]
    
    for_main_points = get_main_points(fid, main_points)[0]
    against_main_points = get_main_points(fid, main_points)[1]
    for_output_list = []
    against_output_list = []
    for sentence in for_main_points:
        for_output_list.append(run_lstm_one_sentence(sentence, title, embedding_index))
    for sentence in against_main_points:
        against_output_list.append(run_lstm_one_sentence(sentence, title, embedding_index))
        
    for_torch = combine_func(torch.stack(for_output_list), dim = 0, keepdim = True)[0]
    against_torch = combine_func(torch.stack(against_output_list), dim = 0, keepdim = True)[0]
    
    return torch.cat((for_, against_), dim = 2)

In [144]:
fid_torch = run_lstm_on_fid(fid, embedding_index, combine_func = torch.max)

In [145]:
fid_torch

tensor([[[ 0.1311,  0.0398, -0.0717, -0.0113,  0.0484,  0.0939,  0.0939,
           0.0785,  0.2296,  0.1391,  0.1186, -0.0312,  0.2409,  0.1822,
           0.2931,  0.1382,  0.1650,  0.0770,  0.1300,  0.3385, -0.0227,
          -0.0478,  0.0381,  0.3026,  0.2034,  0.0691,  0.0747,  0.0797,
          -0.0105,  0.1565,  0.1402,  0.1714,  0.1209,  0.1541,  0.2200,
          -0.0578,  0.0963, -0.0084,  0.0531, -0.0345,  0.1637,  0.0682,
           0.0386,  0.1677,  0.0466,  0.2120, -0.0401,  0.1498,  0.1773,
           0.1450,  0.1453,  0.3394,  0.0595, -0.1048, -0.0602,  0.1921,
           0.1459,  0.0621,  0.1473, -0.0097,  0.0381,  0.0212,  0.1870,
           0.3143, -0.0622, -0.0059,  0.1172,  0.0706,  0.1236,  0.1836,
           0.0090,  0.0752,  0.0059,  0.2280,  0.0627,  0.1630,  0.0061,
           0.3293,  0.3092,  0.1563,  0.1527,  0.1923,  0.0700,  0.2126,
          -0.0039,  0.0276,  0.1177,  0.0073, -0.0256,  0.1523,  0.0495,
           0.1527,  0.0691,  0.0849,  0.0777, -0.00